# Neighborhood graphs

-----

## Setup

Here are loaded the packages used in the rest of notebook:

In [ ]:
use Graph;
use Graph::Circulant;
use Graph::Complete;
use Graph::CompleteKaryTree;
use Graph::Cycle;
use Graph::Grid;
use Graph::Hypercube;
use Graph::KnightTour;
use Graph::Star;
use Graph::Wheel;
use Graph::Path;
use Graph::Petersen;

use Graph::Distribution;
use Graph::Random;

use Data::Reshapers;
use Data::Summarizers;

use JavaScript::D3;
use WWW::MermaidInk;

### JavaScript

Here we prepare the notebook to visualize with JavaScript:

In [ ]:
#% javascript
require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});

Verification:

In [ ]:
#% js
js-d3-list-line-plot(10.rand xx 40, background => 'none', stroke-width => 2)

Here we set a collection of visualization variables:

In [ ]:
my $title-color = 'Ivory';
my $stroke-color = 'SlateGray';
my $tooltip-color = 'LightBlue';
my $tooltip-background-color = 'none';
my $background = '1F1F1F';
my $color-scheme = 'schemeTableau10';
my $edge-thickness = 3;
my $vertex-size = 6;
my $mmd-theme = q:to/END/;
%%{
  init: {
    'theme': 'forest',
    'themeVariables': {
      'lineColor': 'Ivory'
    }
  }
}%%
END
my %force = collision => {iterations => 0, radius => 10},link => {distance => 180};
my %force2 = charge => {strength => -30, iterations => 4}, collision => {radius => 50, iterations => 4}, link => {distance => 30};

------

## Undirected grid graph

In [ ]:
#% js
my $g = Graph::Grid.new(4, 6);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [ ]:
my $g2 = $g.neighborhood-graph(['2_1'], d => 2)

In [ ]:
#% js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## Directed grid graph

In [ ]:
#% js
my $g = Graph::Grid.new(3, 4, :directed);

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [ ]:
my $g2 = $g.neighborhood-graph(['0_0', '1_3'], d => 1)

In [ ]:
$g2.vertex-list

In [ ]:
$g2.edges

In [ ]:
#% js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        directed => $g.directed,
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

------

## In a Petersen graph

In [ ]:
#% js
my $g = Graph::Petersen.new;

$g.edges(:dataset) 
==> js-d3-graph-plot(
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [ ]:
my $g2 = $g.neighborhood-graph(["0", "1"], d => 1)

In [ ]:
#%js
$g.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => [|$g2.vertex-list, |$g2.edges],
        :$background, 
        :$title-color, 
        width => 700, 
        :$edge-thickness,
        :$vertex-size, 
        force => {charge => {strength => -300, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

-----

## Neat examples

### Pick out random neighborhoods from a grid

In [ ]:
my $g = Graph::Grid.new(80, 80);

my $g2 = $g.neighborhood-graph($g.vertex-list.pick(80), d => 1)

In [ ]:
my $nc = $g2.weakly-connected-components.elems;
say "numbef of connected components: $nc";
my @comps = $g2.weakly-connected-components.pick($nc/3);
my @gs = @comps.map({ $g2.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
@highlight.elems

In [ ]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        highlight => { Salmon => @highlight[0 .. @highlight.elems div 2], Moccasin => @highlight[@highlight.elems div 2 .. *]},
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        vertex-color => 'LightSkyBlue',
        edge-color => 'RoyalBlue',
        edge-thickness => 3,
        vertex-size => 4, 
        force => {charge => {strength => -30, iterations => 2}, collision => {radius => 10, iterations => 1}, link => {minDistance => 10}}
    )

In [ ]:
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });
%vertex-coordinates.elems

In [ ]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        vertex-coordinates => ($g2.vertex-list Z=> %vertex-coordinates{$g2.vertex-list}).Hash,
        :$background, 
        title-color => 'none', 
        width => 800, 
        height => 800,
        edge-thickness => 3,
        vertex-size => 0, 
    )

### Subtract random neighborhoods from a grid

In [ ]:
my $g = Graph::Grid.new(60, 30);
my %vertex-coordinates = $g.vertex-list.map({ $_ => $_.split('_')».Int });

my $vset = $g.vertex-list (-) $g.neighborhood-graph($g.vertex-list.pick(60), d => 1).vertex-list;

my $g2 = $g.subgraph($vset.keys)

In [ ]:
my $g3 = $g.neighborhood-graph($g.vertex-list.pick(60), d => 2);
my $nc = $g3.weakly-connected-components.elems;
my @comps = $g3.weakly-connected-components;
my @gs = @comps.map({ $g3.subgraph($_) });
my @highlight = @gs.map({ [|$_.vertex-list, |$_.edges].Slip });
$nc

In [ ]:
#%js
$g2.edges(:dataset) 
==> js-d3-graph-plot(
        :%vertex-coordinates,
        :@highlight,
        highlight-color => 'Ivory',
        :$background, 
        title-color => 'none', 
        width => 1000, 
        height => 500,
        edge-thickness => 4,
        edge-color => 'Gray',
        vertex-size => 0,
        vertex-color => 'Ivory',
        force => {charge => {strength => -30, iterations => 2}, y => {strength => 0.2}, collision => {radius => 1, iterations => 1}, link => {minDistance => 1}}
    )